In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})
mem = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv',
                  infer_datetime_format=True, parse_dates=[5,61,62],
                 dtype= {6:'str',10:'str',11:'str',13:'str'})
npi = pd.read_csv('../output/npi.csv')
npi = npi.drop(columns=['Unnamed: 0'])
print(admis.shape)
print(visits.shape)
print(mem.shape)
print(npi.shape)

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(6213, 23)
(231373, 18)
(61519, 63)
(5415, 3)


In [3]:
visits = pd.merge(visits, npi, how='left', left_on='Service Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Service Provider NPI Type'})
visits = pd.merge(visits, npi, how='left', left_on='Provider Group NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Provider Group NPI Type'})
admis = pd.merge(admis, npi, how='left', left_on='Attending Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Attending Provider NPI Type'})
admis = pd.merge(admis, npi, how='left', left_on='Facility Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Facility Provider NPI Type'})


In [7]:
pcp_visit = 0
readmit = 0
#pull patient numbers that have more then one admission to loop through
for patnum in admis['Patient ID'].drop_duplicates():
    window = pd.to_datetime(0) #set window as a datetime to make first loop work
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = admis[admis['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    vpatient = visits[visits['Patient ID']==patnum].sort_values('Encounter Date')
    dpatient = patient.set_index('Encounter End Date')['Encounter ID']
    dvpatient = vpatient.set_index('Encounter Date')['Encounter ID']
    visittimeline = pd.concat([dpatient ,dvpatient])
    visittimeline.sort_index(inplace=True)
    try:
        if vpatient['Provider Group NPI Type'].value_counts().keys()[0] == 'NPI-1':
            print('NPI-1')
            admis.loc[admis['Patient ID']==patnum,'NPI_type'] = 'NPI-1'
        else:
            admis.loc[admis['Patient ID']==patnum,'NPI_type'] = 'NPI-2'
    except:
        admis.loc[admis['Patient ID']==patnum,'NPI_type'] = 'None'
    
    for rnum in range(0,len(patient)):
        if window < patient.iloc[rnum,8] :
            #test all admissions after rnum
            trigger = patient.iloc[rnum+1:,8]<=(patient.iloc[rnum,9] + pd.DateOffset(days=30)) #column 8=start, 9=end
            #add one if there are Trues
            readmit = readmit + int(trigger.sum() > 0)
            #save 30 day window date to remove dupicate dates
            window = (patient.iloc[rnum,9] + pd.DateOffset(days=30))
            if trigger.sum() > 0:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'readmission'] = True
            else:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
        else:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = False
    for rnum in range(0,len(visittimeline)-1):
        if (visittimeline[rnum].startswith('HIA') & visittimeline[rnum+1].startswith('COV'))&(visittimeline.index[rnum+1] <= (visittimeline.index[rnum])+pd.DateOffset(days=7)):
            pcp_visit += 1
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = True
        elif visittimeline[rnum].startswith('HIA'):
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = False

NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1
NPI-1


In [8]:
admis['index_admis'].value_counts()

True     5536
False     613
Name: index_admis, dtype: int64

In [9]:
(admis['index_admis'].isna()).sum()

64

In [10]:
readmit

527

In [11]:
pcp_visit

1846

In [12]:
#question1
q1num = len(admis[(admis['index_admis']==True)&(admis['readmission']==True)])
q1denom = admis['index_admis'].value_counts()[True]
print('numerator: ' + str(q1num))
print('denominator: ' + str(q1denom))
print(q1num/q1denom)


numerator: 527
denominator: 5536
0.09519508670520231


In [13]:
#question2
q2num = admis['pcp_visit'].value_counts()[True]
q2denom = admis['index_admis'].value_counts().sum()
print('numerator: ' + str(q2num))
print('denominator: ' + str(q2denom))
print(q2num/q2denom)

numerator: 1846
denominator: 6149
0.30021141649048627


In [16]:
admis.NPI_type.value_counts()

NPI-2    5872
None      318
NPI-1      23
Name: NPI_type, dtype: int64